In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pathlib import Path
import re

In [ ]:
# ─────────────────────────────────────────────────────────────────────
# 0  Imports & global settings
# ─────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/processed")           # adjust if the CSVs live elsewhere
WATER_CSV   = DATA_DIR / "water_levels_daily.csv"
WEATHER_CSV = DATA_DIR / "edersee_historical_weather.csv"

# -----------------------------------------------------------------------------#
# 1.  Water‑level series  (1 row per day, continuous)
# -----------------------------------------------------------------------------#
water = (
    pd.read_csv(WATER_CSV, parse_dates=["timestamp"])
      .set_index("timestamp")
      .sort_index()
      .resample("D").mean()             # enforce one value per calendar day
      .interpolate()                    # closes micro‑gaps if any
      .rename(columns={"value": "water_level"})
)

# -----------------------------------------------------------------------------#
# 2.  Calendar (deterministic) features
# -----------------------------------------------------------------------------#
doy = water.index.dayofyear
dow = water.index.dayofweek
water["sin365"] = np.sin(2 * np.pi * doy / 365.25)
water["cos365"] = np.cos(2 * np.pi * doy / 365.25)
water["sin7"]   = np.sin(2 * np.pi * dow / 7)
water["cos7"]   = np.cos(2 * np.pi * dow / 7)

# -----------------------------------------------------------------------------#
# 3.  Weather table — ingest *all* columns, tidy names,
#     resample to daily means and fill ≤7‑day gaps
# -----------------------------------------------------------------------------#
def snake(s: str) -> str:
    """lower‑case & replace any non‑alnum by a single '_'"""
    s = re.sub(r"\([^)]*\)", "", s)      # drop units in (...)
    s = re.sub(r"[^A-Za-z0-9]+", "_", s)
    return re.sub(r"__+", "_", s).strip("_").lower()

weather_raw = (
    pd.read_csv(WEATHER_CSV, skiprows=3, parse_dates=["time"])
      .rename(columns={"time": "timestamp"})
)

weather_raw.columns = [snake(c) for c in weather_raw.columns]

weather = (
    weather_raw.set_index("timestamp")
               .sort_index()
               .resample("D").mean()
               .interpolate(limit=7)
)

# -----------------------------------------------------------------------------#
# 4.  Merge water‑level, calendar & weather
# -----------------------------------------------------------------------------#
daily_df = water.join(weather, how="left")
daily_df = daily_df.interpolate(limit_direction="both")

# -----------------------------------------------------------------------------#
# 5.  Add 28‑day‑ahead **target** column
#     – each row’s target is the water level 28 days after that row’s date
# -----------------------------------------------------------------------------#
HORIZON = 28
daily_df["target_28d"] = daily_df["water_level"].shift(-HORIZON + 1)
daily_df = daily_df.dropna(subset=["target_28d"])   # drop tail rows w/out target

# ─────────────────────────────────────────────────────────────────────
# 6  Efficient lags & rolling windows  (no fragmentation)
# ─────────────────────────────────────────────────────────────────────
NUM_COLS = daily_df.select_dtypes(float).columns.drop("target_28d")

LAGS  = [1, 7, 14, 28]           # days back
ROLLS = [7, 14, 30]              # rolling sums / means windows

# --- 6.1  Build lag snapshots in one go ---------------------------------------
lag_frames = [
    daily_df[NUM_COLS].shift(lag).add_suffix(f"_lag{lag}")
    for lag in LAGS
]

# --- 6.2  Build rolling means / sums in one go --------------------------------
roll_frames = []
for win in ROLLS:
    # mean for every numeric column
    roll_frames.append(
        daily_df[NUM_COLS].rolling(win).mean().add_suffix(f"_mean{win}d")
    )
    # sum only for precip/snow columns
    precip_cols = [c for c in NUM_COLS if "precip" in c or "snow" in c]
    if precip_cols:
        roll_frames.append(
            daily_df[precip_cols].rolling(win).sum().add_suffix(f"_sum{win}d")
        )

# --- 6.3  Concatenate *once* and join back to the master frame ----------------
new_features = pd.concat(lag_frames + roll_frames, axis=1)

# join + drop initial NaNs (from lags/rolls) and defragment with .copy()
daily_df = (
    pd.concat([daily_df, new_features], axis=1)
      .dropna()
      .copy()                   
)

daily_df

,water_level,sin365,cos365,sin7,cos7,snowfall_sum,precipitation_sum,weather_code,precipitation_hours,cloud_cover_mean,...,shortwave_radiation_sum_mean30d,soil_temperature_0_to_100cm_mean_mean30d,sunshine_duration_mean30d,temperature_2m_max_mean30d,temperature_2m_min_mean30d,relative_humidity_2m_mean_mean30d,temperature_2m_mean_mean30d,snowfall_sum_sum30d,precipitation_sum_sum30d,precipitation_hours_sum30d
timestamp,,,,,,,,,,,,,,,,,,,,,
2000-01-30,239.98,0.493468,0.869764,-0.781831,0.623490,0.0,9.9,63.0,16.0,69.0,...,2.598333,3.226667,12200.866000,3.110000,-0.883333,88.233333,1.236667,6.51,52.3,135.0
2000-01-31,241.07,0.508356,0.861147,0.000000,1.000000,0.0,3.4,53.0,15.0,100.0,...,2.613000,3.186667,12200.866000,3.256667,-0.683333,88.033333,1.426667,6.51,53.7,144.0
2000-02-01,241.88,0.523094,0.852275,0.781831,0.623490,0.0,0.0,3.0,0.0,100.0,...,2.629667,3.163333,12094.591667,3.420000,-0.516667,87.800000,1.563333,6.51,53.7,144.0
2000-02-02,242.34,0.537677,0.843151,0.974928,-0.222521,0.0,1.3,53.0,4.0,89.0,...,2.594333,3.150000,11933.581667,3.503333,-0.436667,87.633333,1.656667,6.51,55.0,148.0
2000-02-03,242.58,0.552101,0.833777,0.433884,-0.900969,0.0,3.1,51.0,15.0,70.0,...,2.719000,3.136667,12705.490667,3.490000,-0.436667,87.433333,1.653333,6.51,50.4,145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,242.51,0.989794,0.142508,0.000000,1.000000,0.0,4.6,61.0,12.0,62.0,...,11.605667,3.120000,28813.939333,11.053333,0.570000,74.700000,5.030000,0.00,20.4,81.0
2025-03-25,242.52,0.992099,0.125461,0.781831,0.623490,0.0,0.0,3.0,0.0,59.0,...,11.833667,3.176667,29029.090667,11.040000,0.603333,74.700000,5.053333,0.00,20.4,81.0
2025-03-26,242.51,0.994110,0.108377,0.974928,-0.222521,0.0,0.8,51.0,6.0,73.0,...,12.023667,3.226667,29548.386000,11.006667,0.600000,74.633333,5.050000,0.00,20.9,85.0


In [17]:
daily_df.columns['target_28d']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices